# SplitRec：在隐语拆分学习中使用 FeatureInferenceAttack
在联邦学习中，攻击者可以通过监听训练模型过程中传输的数值和梯度信息，攻击对方模型或数据，在一定程度上推理出有用信息，造成信息泄露。

本文考虑两方拆分学习中的特征推理攻击，将介绍[《Feature Inference Attacks on Model Predictions in Vertical Federated Learning》](https://arxiv.org/abs/2010.10152)中的 GRN 攻击方法在隐语中的使用。


## Feature Inference Attack with Generative Regression Network
特征推理攻击中，有标签的一方作为攻击方，推测对方的特征。在联邦模型训练之后，GRN 攻击方法通过一个生成回归网络（Generator Model）预测对方特征，并通过不断缩小预测特征在联邦模型的输出值和真实联邦模型输出值的差距，训练 Genertor Model，因而可以预测对方特征，如下图所示。

![fia0](resources/fia0.png)

其中 Generator Model 具体训练步骤如下：

1. 将攻击方特征（蓝色）和随机生成的数据（橙色）输入到 Generator Model 中，输出值作为预测的对方特征
2. 将攻击方特征和预测的对方特征输入已经训完的联邦模型中，计算 logit 输出
3. 利用步骤 2 中输出的 logit 与真实 logit（攻击方特征和对方真实特征输入联邦模型计算的 logit）计算损失
4. 对得到的损失进行反向传播，更新 Generator Model 参数

算法伪代码如下：

![fia1](resources/fia1.png)

loss 函数定义如下：

![fia2](resources/fia2.png)

## 隐语中的攻击方法实现
在隐语中攻击方法的实现是通过 callback 机制来完成。攻击算法基类 CallBack 位于 secretflow/ml/nn/sl/backend/torch/callback.py，我们在联邦模型训练的以下几个节点提供 hook，不同攻击方法可以通过将攻击算法实现在对应节点的 hook， 使攻击逻辑注入到联邦模型的训练过程中。

- on_train_begin
- on_train_end
- on_epoch_begin
- on_epoch_end
- on_batch_begin
- on_batch_end

用户如果需要实现自定义的攻击方法，需要

1. 定义 CustomAttacker 继承基类 Callback，将攻击逻辑实现到对应的 hook 函数中
2. 定义 attacker_builder 函数将构建 attacker 写到其中
3. 与普通 Split Learning 模型训练一样定义 sl_model, 并在调用 sl_model.fit() 时，将 callback_dict {party -> attacker_builder} 传入 callbacks 参数即可

其中步骤 1 可以参考隐语中已有的 FeatureInferenceAttacker/LabelInferenceAttacker，步骤 2 和 3 可参考下面 FeatureInferenceAttacker 的使用方式。

## Feature Inferece Attack 的隐语封装
我们在隐语中提供了多种攻击方法的封装。对于论文中的攻击方法，我们提供了 FeatureInferenceAttacker 封装，具体使用可以参考以下代码。

首先和一般 Split Learning 模型训练一样，我们将进行数据处理，并定义一个 SLModel。

然后定义调用 FeatureInferenceAttacker 的 attacker_builder，并在 SLModel fit 时将 attacker_builder 传入进行训练和攻击。 

## 环境设置

In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()
# sf.init(['alice', 'bob'], address="local")
sf.init(['alice', 'bob'], address="local", debug_mode=True)
alice, bob = sf.PYU('alice'), sf.PYU('bob')
device_y = alice

The version of SecretFlow: 1.7.0b0


## 数据集介绍
这里我们使用 UCI Sensorless Drive Diagnosis 数据集，该数据集有 48 维特征 11 分类。

这里我们对数据进行纵向切分，攻击方持有 28 维特征和 label，被攻击方持有 20 维特征。

[数据集官网](http://archive.ics.uci.edu/dataset/325/dataset+for+sensorless+drive+diagnosis)

这里可以下载论文代码数据集： [drive_cleaned.csv](https://raw.githubusercontent.com/xinjianluo/featureinference-vfl/master/datasets/drive_cleaned.csv)

或直接使用我们提供的 demo 数据 drive_cleaned_demo.csv

## 准备数据

In [2]:
# 分割alice和bob的数据集的schema（不是数据集本身）

from secretflow.utils.simulation.datasets import load_bank_marketing

# Alice has the first four features,
# while bob has the left features
data = load_bank_marketing(parts={alice: (0, 4), bob: (4, 16)}, axis=1)
# Alice holds the label.
label = load_bank_marketing(parts={alice: (16, 17)}, axis=1)

from secretflow.preprocessing.scaler import MinMaxScaler
from secretflow.preprocessing.encoder import LabelEncoder

# 类别类编码
encoder = LabelEncoder()
data['job'] = encoder.fit_transform(data['job'])
data['marital'] = encoder.fit_transform(data['marital'])
data['education'] = encoder.fit_transform(data['education'])
data['default'] = encoder.fit_transform(data['default'])
data['housing'] = encoder.fit_transform(data['housing'])
data['loan'] = encoder.fit_transform(data['loan'])
data['contact'] = encoder.fit_transform(data['contact'])
data['poutcome'] = encoder.fit_transform(data['poutcome'])
data['month'] = encoder.fit_transform(data['month'])
label = encoder.fit_transform(label)

# 归一化、 打印数据信息 
scaler = MinMaxScaler()

data = scaler.fit_transform(data)
# data.astype('float32')
# print('type(data)=', type(data))
print(data.columns)
# print(data['job'].loc[0:0])

# print(data[alice].shape)
# print(data)
# print(data.partitions)
# print(data.columns) # alice和bob的数据集的列名（总的列表）
# print(data.partition_columns) # alice和bob的数据集的列名（分别）
# print("data.dtypes: ",data.dtypes)
# print('label.dtypes: ', label.dtypes)

# 转换dtypes
# float64 to float32
float64_colomns = data.select_dtypes(include=['float64']).columns
print(float64_colomns)
data[float64_colomns] = data[float64_colomns].astype('float32')

label=label.astype('float32')

print("data.dtypes: ",data.dtypes)
print("label.dtypes: ",label.dtypes)



# 切分数据集（训练&测试）
from secretflow.data.split import train_test_split

random_state = 1234
train_fea, test_fea, = train_test_split(
    data, train_size=0.8, random_state=random_state
)
train_label, test_label = train_test_split(
    label, train_size=0.8, random_state=random_state
)

print(train_fea.values.partition_shape()[alice][0],) # 总的样本数目

# 获取 data feature mean
mean_attr = data.mean() # 样本均值
# bob_mean = sf.reveal(mean_attr.partitions[bob].data).values
bob_mean = mean_attr[4:].values
print('type(bob_mean): ', type(bob_mean))
# print('bob_mean: ', bob_mean)

INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.


['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
data.dtypes:  {'age': dtype('float32'), 'job': dtype('float32'), 'marital': dtype('float32'), 'education': dtype('float32'), 'default': dtype('float32'), 'balance': dtype('float32'), 'housing': dtype('float32'), 'loan': dtype('float32'), 'contact': dtype('float32'), 'day': dtype('float32'), 'month': dtype('float32'), 'duration': dtype('float32'), 'campaign': dtype('float32'), 'pdays': dtype('float32'), 'previous': dtype('float32'), 'poutcome': dtype('float32')}
label.dtypes:  {'y': dtype('float32')}
3616
type(bob_mean):  <class 'numpy.ndarray'>


/home/dengruijun/miniconda3/envs/drj-sf/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
/home/dengruijun/miniconda3/envs/drj-sf/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


## 定义 SL 模型结构

In [3]:
import torch
import torch.nn as nn
from secretflow.ml.nn.core.torch import BaseModule


class SLBaseNet(nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 100)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(100, output_dim)
        self.relu2 = nn.ReLU()

        self.output_num_value = output_dim
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return x

    def output_num(self): # 学习 secretflow/ml/nn/applications/sl_dnn_torch.py
        return 1

class SLFuseModel(BaseModule):
    def __init__(self,input_dim, output_dim, party_num):
        super().__init__()

        # layers = []
        # for i in range(party_num):
            # layers.append(nn.Linear(input_dim, output_dim))
        nn_layers = []
        self.total_input_dim = input_dim * party_num
        self.fc1 = nn.Linear(self.total_input_dim, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        if isinstance(x, list):
            x = torch.cat(x,dim=1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


## 定义 SL Model

In [4]:
# torch相关
from secretflow.ml.nn.core.torch import (
    metric_wrapper,
    optim_wrapper,
    BaseModule,
    TorchModel,
)
from torchmetrics import Accuracy, Precision
from secretflow.security.aggregation import SecureAggregator
from secretflow.utils.simulation.datasets import load_mnist
from torch import nn, optim
from torch.nn import functional as F
import torch

from secretflow.ml.nn import SLModel

hidden_size = 64

def model_base_alice_fn():
    def create_model():
        return SLBaseNet(input_dim=4, output_dim=hidden_size)
    return create_model

def model_base_bob_fn():
    def create_model():
        return SLBaseNet(input_dim=12, output_dim=hidden_size)
    return create_model

model_base_alice = TorchModel(
    model_fn=model_base_alice_fn(),
    loss_fn=nn.BCELoss,
    optim_fn = optim_wrapper(optim.Adam),
    metrics = [
    metric_wrapper(Accuracy, task='binary',),
    metric_wrapper(Precision, task='binary',),
    ],
    # input_dim=4, output_dim=hidden_size,
    )

model_base_bob = TorchModel(
    model_fn=model_base_bob_fn(),
    loss_fn=nn.BCELoss,
    optim_fn = optim_wrapper(optim.Adam),
    metrics = [
    metric_wrapper(Accuracy, task='binary',),
    metric_wrapper(Precision, task='binary',),
    ],
    # input_dim=12, output_dim=hidden_size,
    )

def model_fuse_fn():
    def create_model():
        return SLFuseModel(input_dim=64, output_dim=1, party_num=2)
    return create_model

fuse_model = TorchModel(
    model_fn=model_fuse_fn(),
    loss_fn=nn.BCELoss,
    optim_fn = optim_wrapper(optim.Adam),
    metrics = [
    metric_wrapper(Accuracy, task='binary',),
    metric_wrapper(Precision, task='binary',),
    ],
    # input_dim=64, output_dim=1, party_num=2,
    )


base_model_dict = {alice: model_base_alice, bob: model_base_bob}

sl_model = SLModel(
    base_model_dict=base_model_dict,
    device_y=device_y,
    model_fuse=fuse_model,
    dp_strategy_dict=None,
    compressor=None,
    simulation=True,
    random_seed=1234,
    backend='torch',
    strategy='split_nn',
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.torch.strategy.split_nn.PYUSLTorchModel'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.torch.strategy.split_nn.PYUSLTorchModel'> with party bob.


## 定义 attacker_builder 


### 2.Model Inversion

In [5]:
# 定义decoder net
class BankNetDecoder(nn.Module):  # cifar10 relu22 decoder网络 （目前结构就完全是和edge net的相反的层）
    def __init__(self, latent_dim=64,target_dim=12):
        super(BankNetDecoder, self).__init__()

        self.delinear1 = nn.Linear(latent_dim, 128)
        self.ReLU1 = nn.ReLU()
        self.delinear2 = nn.Linear(128, 64)
        self.ReLU2 = nn.ReLU()
        self.delinear3 = nn.Linear(64, target_dim)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        _in = x
        for layer in self.children():
            _in = layer(_in)
        return _in

In [6]:
# 用于攻击的数据：
from torch.utils.data import Dataset, DataLoader, TensorDataset

def data_builder(data, label, batch_size):
    def prepare_data():
        # alice_data = data[:, :4]
        # bob_data = data[:, 4:16]

        numpy_alice = sf.reveal(data.partitions[alice].data).values
        numpy_bob = sf.reveal(data.partitions[bob].data).values
        alice_data = torch.tensor(numpy_alice, dtype=torch.float32)
        bob_data = torch.tensor(numpy_bob, dtype=torch.float32)

        # alice_dataset = TensorDataset(torch.tensor(alice_data))
        # alice_dataloader = DataLoader(
        #     dataset=alice_dataset,
        #     shuffle=False,
        #     batch_size=batch_size,
        # )

        bob_dataset = TensorDataset(torch.tensor(bob_data))
        bob_dataloader = DataLoader(
            dataset=bob_dataset,
            shuffle=False,
            batch_size=batch_size,
        )

        dataloader_dict = bob_dataloader
        return dataloader_dict # 返回的是两个一样的，因为攻击者和被攻击者的数据是一样的emm

    return prepare_data

In [7]:
# attack callbacks 创建
import sys
sys.path.append('/home/dengruijun/data/FinTech/Products/secretflow/secretflow/ml/nn/sl/attacks/')

from InverseModel import InverseModelAttackCallbacks


# inverse model
def create_attacker_builder(
    trainloader=None,
    testloader=None,
):
    def attacker_builder():
        victim_model_dict = {
            # 'bob': [SLBaseNet, model_save_path], # bob输入的是10啊？
        }
        optim_fn = optim_wrapper(optim.Adam, lr=0.0001)

        decoder_model = TorchModel(
            model_fn=BankNetDecoder,
            loss_fn=None,
            optim_fn=optim_fn,
            metrics=None,
        )


        attacker = InverseModelAttackCallbacks(
                data_type = 0, # 表格数据
                inverse_dir = '/home/dengruijun/data/FinTech/Products/secretflow/drj-outs/InverseNetwork/inverted/',
                decoder_route = '/home/dengruijun/data/FinTech/Products/secretflow/drj-outs/InverseNetwork/decoder_net.pth',
                device = "cuda:0",
                trainloader= trainloader,
                testloader= testloader,

                attack_party= alice,
                victim_party= bob,
                base_model_list=[alice,bob],
                victim_model_dict=victim_model_dict,
                decoder_model_wrapper=decoder_model,
        )
        return attacker

    return attacker_builder




In [8]:
import os
import shutil

InvNet_path = '/home/dengruijun/data/FinTech/Products/secretflow/drj-outs/InverseNetwork/models/'
if os.path.exists(InvNet_path):
    shutil.rmtree(InvNet_path)
os.mkdir(InvNet_path)
model_save_path = InvNet_path + '/sl_model_victim'
fuse_model_save_path = InvNet_path+'/sl_model_fuse'

batch_size = 64

train_bs = 32
test_bs = 1
trainloader = data_builder(train_fea, train_label, train_bs)()
testloader = data_builder(test_fea, test_label, test_bs)()
print("len(trainloader): ", len(trainloader))
print("len(testloader): ", len(testloader))

print("trainloader: ", trainloader)

# for trn_x in testloader:
#     print(trn_x[0].shape)

callback_dict = [create_attacker_builder(
    trainloader,
    testloader,
    )()]



len(trainloader):  113
len(testloader):  905
trainloader:  <torch.utils.data.dataloader.DataLoader object at 0x7f9a8b99a800>
trainloader in callbacks:  <torch.utils.data.dataloader.DataLoader object at 0x7f9a8b99a800>
testloader in callbacks:  <torch.utils.data.dataloader.DataLoader object at 0x7f9a8b99a590>


/tmp/ipykernel_383139/122837605.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bob_dataset = TensorDataset(torch.tensor(bob_data))


## 开始训练和攻击

In [9]:
sl_model.fit(
    train_fea,
    train_label,
    validation_data=(test_fea, test_label),
    epochs=1,
    batch_size=batch_size,
    shuffle=False,
    random_seed=1234,
    dataset_builder=None,
    callbacks=callback_dict, # 暂时注释掉，callback完成后恢复 @caibei
)

INFO:root:SL Train Params: {'self': <secretflow.ml.nn.sl.sl_model.SLModel object at 0x7f9a8d99af20>, 'x': VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f9b3b15ab90>, PYURuntime(bob): <secretflow.data.core.partition.Partition object at 0x7f9b3b15b370>}, aligned=True), 'y': VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f9b3b15bc70>}, aligned=True), 'batch_size': 64, 'epochs': 1, 'verbose': 1, 'callbacks': [<InverseModel.InverseModelAttackCallbacks object at 0x7f9a8b99ac50>], 'validation_data': (VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f9b3b15b100>, PYURuntime(bob): <secretflow.data.core.partition.Partition object at 0x7f9b3b15b580>}, aligned=True), VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f9b3b15be20>}, aligned=True)), 'shuffle': False, 'sample_weight': None, 'validation_freq':

decoder_net:  BankNetDecoder(
  (delinear1): Linear(in_features=64, out_features=128, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=128, out_features=64, bias=True)
  (ReLU2): ReLU()
  (delinear3): Linear(in_features=64, out_features=12, bias=True)
  (sigmoid): Sigmoid()
)
----train decoder----
client_net: 
GeneratedTorchModule(
  (fc1): Linear(in_features=12, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=64, bias=True)
  (relu2): ReLU()
)
decoder_net: 
BankNetDecoder(
  (delinear1): Linear(in_features=64, out_features=128, bias=True)
  (ReLU1): ReLU()
  (delinear2): Linear(in_features=128, out_features=64, bias=True)
  (ReLU2): ReLU()
  (delinear3): Linear(in_features=64, out_features=12, bias=True)
  (sigmoid): Sigmoid()
)


100%|██████████| 905/905 [00:01<00:00, 904.73it/s] 


average cos: 0.9883629339834603
average euc: 0.17577367256194848
average mse: 0.0040450720415895714
average time: 0.0004304164022371914 avg infer time:0.00023147203645653488
inverse done


{'train_loss': [array(0.36142364, dtype=float32)],
 'train_BinaryAccuracy': [array(0.8420907, dtype=float32)],
 'train_BinaryPrecision': [array(0.08955224, dtype=float32)],
 'val_loss': [array(0.17533527, dtype=float32)],
 'val_BinaryAccuracy': [array(0.8729282, dtype=float32)],
 'val_BinaryPrecision': [array(0., dtype=float32)]}

In [10]:
# 保存训练好的base model？
sl_model.save_model(base_model_path=model_save_path,
                    fuse_model_path=fuse_model_save_path)

## 总结
本文通过 UCI Sensorless Drive Diagnosis 数据集上的特征攻击任务来演示了如何通过隐语来使用 FeatureInferenceAttack。

您可以：

1. 下载并拆分数据集，准备训练、攻击使用的数据
2. 定义拆分模型结构及 SL Model
3. 定义 attacker_builder，在其中定义攻击需要的 data_builder 和 FeatureInfereceAttacker
4. 调用 SL Model 进行训练攻击

您可以在自己的数据集上进行尝试，如有任何问题，可以在 github 进行训练即可。